#  FrozenLake
Today you are going to learn how to survive walking over the (virtual) frozen lake through discrete optimization.

<img src="http://vignette2.wikia.nocookie.net/riseoftheguardians/images/4/4c/Jack's_little_sister_on_the_ice.jpg/revision/latest?cb=20141218030206" alt="a random image to attract attention" style="width: 400px;"/>


In [394]:
import gym

#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

[2017-04-28 17:31:47,159] Making new env: FrozenLake-v0


In [395]:
# display the game state
env.render()

SFFF
FHFH
FFFH
HFFG



In [396]:
#create a single game instance
env = gym.make("FrozenLake-v0")

#start new game
env.reset();

[2017-04-28 17:31:49,059] Making new env: FrozenLake-v0


### legend

![img](https://cdn-images-1.medium.com/max/800/1*MCjDzR-wfMMkS0rPqXSmKw.png)

### Gym interface

The three main methods of an environment are
* __reset()__ - reset environment to initial state, _return first observation_
* __render()__ - show current environment state (a more colorful version :) )
* __step(a)__ - commit action __a__ and return (new observation, reward, is done, info)
 * _new observation_ - an observation right after commiting the action __a__
 * _reward_ - a number representing your reward for commiting action __a__
 * _is done_ - True if the MDP has just finished, False if still in progress
 * _info_ - some auxilary stuff about what just happened. Ignore it for now

In [397]:
print("initial observation code:", env.reset())
print('printing observation:')
env.render()
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)

('initial observation code:', 0)
printing observation:
SFFF
FHFH
FFFH
HFFG

('observations:', Discrete(16), 'n=', 16)
('actions:', Discrete(4), 'n=', 4)


In [398]:
env.step(2)

(4, 0.0, False, {'prob': 0.3333333333333333})

In [399]:
env.render()

SFFF
FHFH
FFFH
HFFG
  (Right)


In [401]:
print("taking action 2 (right)")
new_obs, reward, is_done, _ = env.step(3)
print("new observation code:", new_obs)
print("reward:", reward)
print("is game over?:", is_done)
print("printing new state:")
env.render()

taking action 2 (right)
('new observation code:', 5)
('reward:', 0.0)
('is game over?:', True)
printing new state:
SFFF
FHFH
FFFH
HFFG
  (Up)


In [402]:
action_to_i = {
    'left':0,
    'down':1,
    'right':2,
    'up':3
}

### Play with it
* Try walking 5 steps without falling to the (H)ole
 * Bonus quest - get to the (G)oal
* Sometimes your actions will not be executed properly due to slipping over ice
* If you fall, call __env.reset()__ to restart

In [403]:
env.reset()
env.step(action_to_i['down'])
env.step(action_to_i['down'])
env.step(action_to_i['right'])
env.step(action_to_i['right'])
env.step(action_to_i['down'])
env.step(action_to_i['right'])
env.render()

SFFF
FHFH
FFFH
HFFG
  (Right)


### Policy

* The environment has a 4x4 grid of states (16 total), they are indexed from 0 to 15
* From each states there are 4 actions (left,down,right,up), indexed from 0 to 3

We need to define agent's policy of picking actions given states. Since we have only 16 disttinct states and 4 actions, we can just store the action for each state in an array.

This basically means that any array of 16 integers from 0 to 3 makes a policy.

In [465]:
import numpy as np
n_states = env.observation_space.n
n_actions = env.action_space.n
def get_random_policy(n_actions=4, n_states=16):
    """
    Build a numpy array representing agent policy.
    This array must have one element per each of 16 environment states.
    Element must be an integer from 0 to 3, representing action
    to take from that state.
    """
    return np.random.randint(0,n_actions,n_states)

In [405]:
np.random.seed(1234)
policies = [get_random_policy() for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

('Action frequencies over 10^4 samples:', array([ 0.25014375,  0.25130625,  0.2495375 ,  0.2490125 ]))
Seems fine!


### Let's evaluate!
* Implement a simple function that runs one game and returns the total reward

In [406]:
def sample_reward(env, policy, t_max=100):
    """
    Interact with an environment, return sum of all rewards.
    If game doesn't end on t_max (e.g. agent walks into a wall), 
    force end the game and return whatever reward you got so far.
    Tip: see signature of env.step(...) method above.
    """
    s = env.reset()
    total_reward = 0
    
    for _ in range(t_max):
        action = policy[s]
        s,r,done,_=env.step(action)
        total_reward+= r
        if (done):
            break
    return total_reward

In [407]:
print("generating 10^3 sessions...")
rewards = [sample_reward(env,get_random_policy()) for _ in range(10**3)]
assert all([type(r) in (int, float) for r in rewards]), 'sample_reward must return a single number'
assert all([0 <= r <= 1 for r in rewards]), 'total rewards should be between 0 and 1 for frozenlake (if solving taxi, delete this line)'
print("Looks good!")

generating 10^3 sessions...
Looks good!


In [408]:
def evaluate(policy, n_times=100):
    """Run several evaluations and average the score the policy gets."""
    rewards = [sample_reward(env,policy) for _ in range(n_times)]
    return float(np.mean(rewards))
        

In [409]:
def print_policy(policy):
    """a function that displays a policy in a human-readable way."""
    lake = "SFFFFHFHFFFHHFFG"
    assert env.spec.id == "FrozenLake-v0", "this function only works with frozenlake 4x4"

    
    # where to move from each tile
    arrows = ['<v>^'[a] for a in policy]
    
    #draw arrows above S and F only
    signs = [arrow if tile in "SF" else tile for arrow, tile in zip(arrows, lake)]
    
    for i in range(0, 16, 4):
        print(' '.join(signs[i:i+4]))

print("random policy:")
print_policy(get_random_policy())

random policy:
> < < >
v H > H
> ^ > H
H > < G


### Random search

In [410]:
import tqdm

In [411]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(10000)):
    policy = get_random_policy()
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)
        print("Best policy:")

print_policy(best_policy)

  0%|          | 12/10000 [00:00<03:17, 50.48it/s]

('New best score:', 0.0)
Best policy:
('New best score:', 0.04)
Best policy:
('New best score:', 0.13)
Best policy:


  1%|          | 70/10000 [00:01<02:26, 67.56it/s]

('New best score:', 0.19)
Best policy:


  3%|▎         | 270/10000 [00:03<02:24, 67.57it/s]

...

('New best score:', 0.62)
Best policy:


 79%|███████▉  | 7911/10000 [02:17<00:30, 67.51it/s]

('New best score:', 0.77)
Best policy:


100%|██████████| 10000/10000 [02:43<00:00, 61.14it/s]

< < < <
< H > H
^ v < H
H > v G


# Part II Genetic algorithm

The next task is to devise some more effecient way to perform policy search.
We'll do that with a bare-bones evolutionary algorithm.
[unless you're feeling masochistic and wish to do something entirely different which is bonus points if it works]

In [445]:
def crossover(policy1, policy2, p=0.5): #для каждого действия с вер р влево, с вер 1-р вправо
    #return [np.random.choice([policy1[i], policy2[i]], p=[p, 1-p]) for i in range(0,n_states)]
    return  np.array(np.choose(np.random.choice(np.array([0,1]), size=policy1.shape, p=[p, 1-p]), choices=[policy1, policy2]))
        

In [478]:
def mutation(policy, p=0, n_actions=4, n_states=16):
    """
    for each state, with probability p replace action with random action
    Tip: mutation can be written as crossover with random policy
    """
    return crossover(policy, get_random_policy(n_actions, n_states), p=1-p)
    

In [456]:
import numpy as np
np.random.seed(1234)
policies = [crossover(get_random_policy(), get_random_policy()) 
            for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [457]:

n_epochs = 20 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 100 #how many crossovers to make on each step
n_mutations = 0 #how many mutations to make on each tick


In [458]:
print("initializing...")
pool = [get_random_policy() for i in range(pool_size)]
pool_scores = [evaluate(p) for p in pool]


initializing...


In [459]:
assert type(pool) == type(pool_scores) == list
assert len(pool) == len(pool_scores) == pool_size
assert all([type(score) in (float, int) for score in pool_scores])


In [460]:
from random import choice
choice(pool)

array([3, 2, 0, 2, 3, 1, 2, 0, 0, 3, 2, 3, 2, 1, 1, 2])

In [463]:
# uniform priorities
priorities = [ 1./n_crossovers for _ in range(n_crossovers)]
mutation_proba_max = 0.1

#main loop
for epoch in range(n_epochs):
    mutation_proba = mutation_proba_max * (n_epochs - epoch - 1) / n_epochs
    
    print("Epoch: {}, mutation proba {:.3f}:".format(epoch, mutation_proba))
    
    cross_ids1 = np.random.choice(range(n_crossovers), size=n_crossovers, p=priorities)
    cross_ids2 = np.random.choice(range(n_crossovers), size=n_crossovers, p=priorities)
    
    #print(cross_ids1)
    
    #crossovered = [crossover(choice(pool), choice(pool)) for _ in range(n_crossovers)]
    crossovered = [crossover(pool[id1], pool[id2]) for (id1, id2) in zip(cross_ids1, cross_ids2)]
    #mutated = [mutation(choice(pool)) for _ in range(n_mutations)]
    
    #crossovered = [crossover(choice(pool), choice(pool)) for _ in range(pool_size)]
    crossovered_mutated = [mutation(item, p=mutation_proba) for item in crossovered]
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    #pool = pool + crossovered + mutated
    pool = pool + crossovered_mutated
    pool_scores = [evaluate(p) for p in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]
    
    # set priorities
    pool_scores_arr = np.array(pool_scores)
    priorities = list(pool_scores_arr / sum(pool_scores_arr))
    
    ####print(pool_scores)
    #print(sum(pool_scores))
    #print(np.array(pool_scores) / sum(pool_scores))
    
    #print(priorities)

    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])

print_policy(pool[-1])

Epoch: 0, mutation proba 0.095:
('best score:', 0.19)
Epoch: 1, mutation proba 0.090:
('best score:', 0.33)
Epoch: 2, mutation proba 0.085:
('best score:', 0.31)
Epoch: 3, mutation proba 0.080:
('best score:', 0.56)
Epoch: 4, mutation proba 0.075:
('best score:', 0.63)
Epoch: 5, mutation proba 0.070:
('best score:', 0.74)
Epoch: 6, mutation proba 0.065:
('best score:', 0.74)
Epoch: 7, mutation proba 0.060:
('best score:', 0.76)
Epoch: 8, mutation proba 0.055:
('best score:', 0.78)
Epoch: 9, mutation proba 0.050:
('best score:', 0.82)
Epoch: 10, mutation proba 0.045:
('best score:', 0.84)
Epoch: 11, mutation proba 0.040:
('best score:', 0.79)
Epoch: 12, mutation proba 0.035:
('best score:', 0.84)
Epoch: 13, mutation proba 0.030:
('best score:', 0.83)
Epoch: 14, mutation proba 0.025:
('best score:', 0.82)
Epoch: 15, mutation proba 0.020:
('best score:', 0.83)
Epoch: 16, mutation proba 0.015:
('best score:', 0.81)
Epoch: 17, mutation proba 0.010:
('best score:', 0.84)
Epoch: 18, mutation 

## moar

The parameters of the genetic algorithm aren't optimal, try to find something better. (size, crossovers and mutations)

Try alternative crossover and mutation strategies
* prioritize crossover for higher-scorers?
* try to select a more diverse pool, not just best scorers?
* Just tune the f*cking probabilities.

See which combination works best!

# Part III

The frozenlake problem above is just too simple: you can beat it even with a random policy search. Go solve something more complicated.

* __FrozenLake8x8-v0__ - frozenlake big brother (achieve score >0.7)
* __Taxi-v1__ - essentially a maze where you get score for moving passengers to their destinations. (achieve score >5)


### Some tips:
* Random policy search is worth trying as a sanity check, but in general you should expect the genetic algorithm (or anything you devised in it's place) to fare much better that random.
* While _it's okay to adapt the tabs above to your chosen env_, make sure you didn't hard-code any constants there (e.g. 16 states or 4 actions).
* `print_policy` function was built for the frozenlake-v0 env so it will break on any other env. You could simply ignore it OR write your own visualizer for bonus points.
* in function `sample_reward`, __make sure t_max steps is enough to solve the environment__ even if agent is sometimes acting suboptimally. To estimate that, run several sessions without time limit and measure their length.

### Bonus I:
* Gym envs have a condition for "beating the game". E.g. here's the conditions for [Taxi-v1](https://gym.openai.com/envs/Taxi-v1). 
 * If you managed to do that, it's worth uploading your first solution to gym. See `gym.upload(...)` docs. Allbeit it isn't a strong AI (or is it?), uploading your algorithm would be a good start. (and a +point!)

### Bonus II:
* There are environments with continuous state spaces. In fact, most real world environments have this property. While we will dive into methods designed for that later, right now you already can solve them through binarization.
 * Gym has a basic infinite-state-space env called [CartPole](https://gym.openai.com/envs/CartPole-v0) - please start from this one. Solving something more challenging is great, but make sure your algorithm beats cartpole first. Also kudos for submitting.
 * Main idea: if you have something infinite and you want something discrete, you split it into bins. Like what histogram does.
 * Good choice of discretes is critical!
 * If the dimensionality is too high, you can try to reduce it (PCA/autoencoders)



In [467]:
# create Frozen Lake 8x8 instance
env = gym.make("FrozenLake8x8-v0")

[2017-04-28 18:31:07,201] Making new env: FrozenLake8x8-v0


In [468]:
print("observations:", env.observation_space, 'n=', env.observation_space.n)
print("actions:", env.action_space, 'n=', env.action_space.n)
env.render()

('observations:', Discrete(64), 'n=', 64)
('actions:', Discrete(4), 'n=', 4)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG



## Get random policies

In [469]:
n_states = env.observation_space.n
n_actions = env.action_space.n

In [470]:
np.random.seed(1234)
policies = [get_random_policy(n_actions, n_states) for i in range(10**4)]
assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should match n_actions-1'
action_probas = np.unique(policies, return_counts=True)[-1] /10**4. /n_states
print("Action frequencies over 10^4 samples:",action_probas)
assert np.allclose(action_probas, [1. / n_actions] * n_actions, atol=0.05), "The policies aren't uniformly random (maybe it's just an extremely bad luck)"
print("Seems fine!")

('Action frequencies over 10^4 samples:', array([ 0.24981406,  0.25094844,  0.24953906,  0.24969844]))
Seems fine!


## Random search

In [471]:
best_policy = None
best_score = -float('inf')

from tqdm import tqdm
for i in tqdm(range(10000)):
    policy = get_random_policy(n_actions, n_states)
    score = evaluate(policy)
    if score > best_score:
        best_score = score
        best_policy = policy
        print("New best score:", score)
        print("Best policy:")

  0%|          | 2/10000 [00:00<15:30, 10.74it/s]

('New best score:', 0.0)
Best policy:


  0%|          | 6/10000 [00:00<12:56, 12.87it/s]

('New best score:', 0.02)
Best policy:


  1%|          | 57/10000 [00:02<06:48, 24.34it/s]

...

('New best score:', 0.06)
Best policy:


  1%|▏         | 125/10000 [00:05<06:58, 23.61it/s]

('New best score:', 0.09)
Best policy:


100%|██████████| 10000/10000 [07:50<00:00, 21.26it/s]


AssertionError: this function only works with frozenlake 4x4

## Genetic algorithm search

In [474]:
import numpy as np
np.random.seed(1234)
policies = [crossover(get_random_policy(n_actions, n_states), get_random_policy(n_actions, n_states)) 
            for i in range(10**4)]

assert all([len(p) == n_states for p in policies]), 'policy length should always be 16'
assert np.min(policies) == 0, 'minimal action id should be 0'
assert np.max(policies) == n_actions-1, 'maximal action id should be n_actions-1'

assert any([np.mean(crossover(np.zeros(n_states), np.ones(n_states))) not in (0, 1)
               for _ in range(100)]), "Make sure your crossover changes each action independently"
print("Seems fine!")

Seems fine!


In [481]:

n_epochs = 30 #how many cycles to make
pool_size = 100 #how many policies to maintain
n_crossovers = 100 #how many crossovers to make on each step
n_mutations = 0 #how many mutations to make on each tick


In [482]:
print("initializing...")
pool = [get_random_policy(n_actions, n_states) for i in range(pool_size)]
pool_scores = [evaluate(p) for p in pool]


initializing...


In [483]:
# uniform priorities
priorities = [ 1./n_crossovers for _ in range(n_crossovers)]
mutation_proba_max = 0.1

#main loop
for epoch in range(n_epochs):
    mutation_proba = mutation_proba_max * (n_epochs - epoch - 1) / n_epochs
    
    print("Epoch: {}, mutation proba {:.3f}:".format(epoch, mutation_proba))
    
    cross_ids1 = np.random.choice(range(n_crossovers), size=n_crossovers, p=priorities)
    cross_ids2 = np.random.choice(range(n_crossovers), size=n_crossovers, p=priorities)

    crossovered = [crossover(pool[id1], pool[id2]) for (id1, id2) in zip(cross_ids1, cross_ids2)]
    crossovered_mutated = [mutation(item, p=mutation_proba, n_actions=n_actions, n_states=n_states) for item in crossovered]
    
    assert type(crossovered) == type(mutated) == list
    
    #add new policies to the pool
    pool = pool + crossovered_mutated
    pool_scores = [evaluate(p) for p in pool]
    
    #select pool_size best policies
    selected_indices = np.argsort(pool_scores)[-pool_size:]
    pool = [pool[i] for i in selected_indices]
    pool_scores = [pool_scores[i] for i in selected_indices]
    
    # set priorities
    pool_scores_arr = np.array(pool_scores)
    priorities = list(pool_scores_arr / sum(pool_scores_arr))
    
    print(pool_scores)
    
    #print the best policy so far (last in ascending score order)
    print("best score:", pool_scores[-1])

Epoch: 0, mutation proba 0.097:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.02, 0.02, 0.02, 0.03, 0.03, 0.05]
('best score:', 0.05)
Epoch: 1, mutation proba 0.093:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.0